<a href="https://colab.research.google.com/github/FredArgoX/ChaoticTest_PySpark/blob/main/01_GL_Spark_DataFrame_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:

- A much simpler syntax
- Ability to use SQL directly in the dataframe
- Operations are automatically distributed across RDDs

If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowlwdge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those of other programs is that Spark cand handle data across many RDDs, huge data sets that would never fit on a single computer.

# Creating a DataFrame

First we need to start a SparkSession:

In [1]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [2]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [3]:
spark

You will first need to get the data from a file (or connect to a large distributed file like HDFS)

In [ ]:
# This dataset is from Spark's examples